# Playing a Game with Neural Networks

## Simulating game environments

In [1]:
!pip3 install h5py # Esta libreria la ocupa keras-lr
!pip3 install gym # gym es una herramienta para el desarrollo y comparacion 
    #de algoritmos de aprendizade por reforzamiento, contiene distintos
    #entornos para el entrenamiento de estos algoritmos
#!conda install -c menpo ffmpeg
# como no uso conda, busqué la alternativa que fue muy simple, menpo y ffmpeg-python
!pip3 install ffmpeg-python #libreria para usar codecs de video
!pip3 install cython #libreria necesaria para instalar menpo, permite llamar funciones en c 
        #y usar codigo en cython
!pip3 install menpo #se usa como una libreria para el preprocesamiento en muchas tareas
        # de vision artificial.

In [2]:
import numpy as np 
import gym

In [3]:
env = gym.make('CartPole-v0') #Es el típico problema del péndulo invertido sobre un carro
np.random.seed(42), env.seed(42) #establece la semilla para la generacón de numeros
        #pseudoaleatorios para numpy y el carro.
nb_actions = env.action_space.n #Obtiene el numero de acciones que puede hacer el agente, en este caso
        #solo son 2 valores, 0 (mover a la izquierda) y 1 (mover a la derecha). 
input_shape = (1, env.observation_space.shape[0]) #Obtiene la forma de la entrada, dada por el espacio de
        #observación del agente, este es un arreglo de 4 flotantes, 2 de estos dan la posición del carro
        #y los otros 2 la posición angular y velocidad de la polea.
input_shape

(1, 4)

In [4]:
observation = env.reset() #reinicia el agente para que esté en su posición inicial
for t in range(200): #ciclo for donde el agente realizará algunas acciones
    env.render() #renderiza el entorno
    act = env.action_space.sample() #obtiene la muestra de la primera accion
    obs, rwrd, done, info = env.step(act) #realiza la acción 
    if done: #este if comprueba si terminó el "juego"
        print("Episode concluded after %i timesteps" % (t+1)) #cuando termina muestra cuantos ciclos pasaron
                #antes de terminar
        break
env.close() #termina este entorno.

Episode concluded after 18 timesteps


## Presenting Q-learning

In [5]:
!pip3 install keras-rl2 #aqui tuve que usar keras-rl2 porque la libreria original la dejaron de actualizar
        #y ya no funciona con tensorflow 2.0, como observación, para hacer funcionar esta tuve que deinstalar
        #tanto keras-rl como keras, al volver a instalar keras e instalar keras-rl2 ya funcionó bien el código

In [6]:
 #librerias para la creacion de la red neuronal
from keras.models import Sequential
from keras.layers import Dense, Activation 
from keras.layers import Flatten, Dropout 
from keras.optimizers import Adam
#librerias para el agente y el aprendizaje por reforzamiento
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [7]:
#creación del modelo
model = Sequential() #es un modelo de manera secuencial
model.add(Flatten(input_shape=input_shape)) #capa de entrada del tamaño que se obtuvo segun los posibles
        #movimientos del agente
model.add(Dense(12)) #capa densa con 12 neuronas
model.add(Activation('relu')) #función de activación ReLu
model.add(Dense(nb_actions)) #otra capa densa con el valor de acciones que puede hacer el agente,
        #es decir, da la posible respuesta del agente
model.add(Activation('linear')) #capa de activación linear

print(model.summary()) #imprime el resumen del modelo

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 12)                60        
_________________________________________________________________
activation (Activation)      (None, 12)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 86
Trainable params: 86
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
policy = EpsGreedyQPolicy(eps=0.3) #Politica de avaricia epsilon, este toma una accion aleatoria para
        #poder explorar los resultados, y poder escoger la accion que da la mejor recompenza
        #con prueba y error.
memory = SequentialMemory(limit=50000, #memoria del agente
                          window_length=1)

dqn = DQNAgent(model=model, #Deep Q Network Agent, esta toma el modelo de la red creado anteriormente
               nb_actions=nb_actions,  #las acciones
               memory=memory, #la memoria
               nb_steps_warmup=50, #Al investigar sobre este parametro, encontré que en las primeras
                   #iteraciones del agente este obtendrá errores muy largos, por lo que este parametro
                   #ayuda a tener un "calentamiento" gradual, lo veo como un tipo de "ratio de aprendizaje"
               target_model_update=0.01, #es un ratio de actualización del modelo, si es un número mayor a 1
                   #indica cada cuantos pasos el modelo se actualiza (hard-update), si es menor a 1, se 
                   #le llama soft-update el cual lo actualiza en el porcentaje dado, de igual manera
                   #le veo similitud al ratio de aprendizaje, la diferencia con el parametro radica en que
                   #este modifica la red, y el anterior tiene que ver con los ciclos que realiza el agente
                   #antes de modificar la red
               policy=policy)#politica de aprendizaje

dqn.compile(Adam(lr=0.001)) #compila el modelo y establece el ratio de aprendizaje de este

training = dqn.fit(env, nb_steps=30000, #entrena el agente con el modelo, establece 30000 ciclos
                   visualize=False, verbose=1) 

Training for 30000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10000/10000 [==============================] - 45s 5ms/step - reward: 1.0000
294 episodes - episode_reward: 33.378 [8.000, 180.000] - loss: 3.040 - mean_q: 17.694

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 1.0000
58 episodes - episode_reward: 173.103 [25.000, 200.000] - loss: 8.489 - mean_q: 44.859

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: 1.0000
done, took 140.388 seconds


In [9]:
env = gym.make('CartPole-v0') #crea un nuevo agente de pendulo invertido sobre el carro
mon = gym.wrappers.Monitor(env, #crea un monitor para visualizar el experimento, este crea un video
                           "./gym-results", #en la carpeta dada como parametro
                           force=True)
mon.reset() #reinicia el monitor
dqn.test(mon, nb_episodes=1, visualize=True) #inicia la prueba con la red neuronal y el monitor
mon.close() #cierra el monitor
env.close() #cierra el agente

Testing for 1 episodes ...
Episode 1: reward: 200.000, steps: 200


In [10]:
#El siguiente código sirve para visualizar el video en el notebook.
import io
import base64
from IPython.display import HTML

template = './gym-results/openaigym.video.%s.video000001.mp4'
video = io.open(template % mon.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
<video width="520" height="auto" alt="test" controls>
<source src="data:video/mp4;base64,{0}"
 type="video/mp4" />
</video>'''.format(encoded.decode('ascii')))